In [1]:
import pandas as pd
import numpy as np 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, html, dcc, callback 
from dash.dependencies import Input, Output 
from skimage import io
import zarr 

In [17]:
##Importing the data

track_df = pd.read_pickle('/Users/apple/Desktop/Akamatsu_Lab/Lap_track/Final/test_data/datasets/track_df_cleaned_final_full.pkl')
filtered_tracks = pd.read_pickle('/Users/apple/Desktop/Akamatsu_Lab/Lap_track/Final/test_data/datasets/filtered_tracks_final.pkl')

zarr_arr = zarr.open(store = '/Users/apple/Desktop/Akamatsu_Lab/Lap_track/Final/test_data/zarr_file/all_channels_data', mode = 'r')

##Generate the unique number of tracks 

unique_tracks = filtered_tracks['track_id'].unique()

In [18]:
track_df.columns

Index(['frame', 'c3_gaussian_amp', 'c3_mu_x', 'c3_mu_y', 'c3_mu_z',
       'c3_sigma_x', 'c3_sigma_y', 'c3_sigma_z', 'tree_id', 'track_id',
       'c3_mean_amp', 'c3_voxel_sum', 'c3_voxel_sum_adjusted', 'c3_peak_amp',
       'c3_peak_x', 'c3_peak_y', 'c3_peak_z', 'c2_amp', 'c2_peak_amp',
       'c2_peak_x', 'c2_peak_y', 'c2_peak_z', 'c2_peak_mean', 'c2_voxel_sum',
       'c2_voxel_sum_adjusted', 'c2_gaussian_amp', 'c2_mu_x', 'c2_mu_y',
       'c2_mu_z', 'c2_sigma_x', 'c2_sigma_y', 'c2_sigma_z', 'c1_amp',
       'c1_peak_amp', 'c1_peak_x', 'c1_peak_y', 'c1_peak_z', 'c1_voxel_sum',
       'c1_voxel_sum_adjusted', 'c1_gaussian_amp', 'c1_mu_x', 'c1_mu_y',
       'c1_mu_z', 'c1_sigma_x', 'c1_sigma_y', 'c1_sigma_z'],
      dtype='object')

In [3]:
zarr_arr.shape

(130, 3, 75, 258, 275)

In [4]:
filtered_tracks.head()

,track_id,track_length,track_start,track_end,c3_peak,c2_peak,c1_peak,c3_peak_frame,c2_peak_frame,c1_peak_frame,mean_displacement,mean_z,mean_z_displacement,dnm2_positive,actin_positive,membrane_region
22,22,8,0,7,268.426824,168.0,238.0,4,7,6,1.080068,1.625000,0.428571,False,False,Basal
65,65,8,0,7,345.835302,224.0,420.0,6,6,6,2.043674,6.625000,1.571429,True,True,Basal
93,93,8,0,7,308.000000,169.0,230.0,6,3,5,1.222925,11.500000,0.857143,False,False,Lateral
95,95,8,0,7,332.333333,201.0,251.0,6,7,7,3.090703,13.000000,1.714286,False,False,Lateral
100,100,22,0,21,257.000000,188.0,513.0,13,21,13,2.233344,12.090909,0.904762,False,True,Lateral


In [7]:
def select_type_of_tracks(dataframe, only_dynamin_tracks = False, only_actin_tracks = False, all_positive_tracks = True, both_tracks_positive = False): 
        if only_dynamin_tracks == True: 
            relevant_tracks = dataframe[(dataframe['dnm2_positive'] == True) & (dataframe['actin_positive'] == False)]['track_id'].values
        elif only_actin_tracks == True: 
            relevant_tracks = dataframe[(dataframe['dnm2_positive'] == False) & (dataframe['actin_positive'] == True)]['track_id'].values
        elif all_positive_tracks == True: 
            relevant_tracks = dataframe[(dataframe['dnm2_positive'] == True) | (dataframe['actin_positive'] == True)]['track_id'].values
        elif both_tracks_positive == True: 
            relevant_tracks = dataframe[(dataframe['dnm2_positive'] == True) & (dataframe['actin_positive'] == True)]['track_id'].values

        return relevant_tracks

In [8]:
def select_tracks_region_wise(dataframe, tracks, only_basal, only_apical, only_lateral, all):
    updated_tracks_df = dataframe[dataframe['track_id'].isin(tracks)]
    if only_basal == True:
        updated_tracks = updated_tracks_df[updated_tracks_df['membrane_region'] == 'Basal']['track_id'].values
    elif only_apical == True: 
        updated_tracks = updated_tracks_df[updated_tracks_df['membrane_region'] == 'Apical']['track_id'].values
    elif only_lateral == True: 
        updated_tracks = updated_tracks_df[updated_tracks_df['membrane_region'] == 'Lateral']['track_id'].values
    elif all == True: 
        return updated_tracks_df['track_id'].values 
    
    return updated_tracks

In [19]:
def select_type_of_intensity(type, voxel_sum_col_names = ['c3_voxel_sum', 'c2_voxel_sum', 'c1_voxel_sum'], 
                             adjusted_voxel_sum_col_names = ['c3_voxel_sum_adjusted', 'c2_voxel_sum_adjusted', 'c1_voxel_sum_adjusted'], 
                             gaussian_col_names = ['c3_gaussian_amp', 'c2_gaussian_amp', 'c1_gaussian_amp'], 
                             peak_col_names = ['c3_peak_amp', 'c2_peak_amp', 'c1_peak_amp']): 
    if type == 'Adjusted Voxel Sum': 
        return adjusted_voxel_sum_col_names
    elif type == 'Voxel Sum': 
        return voxel_sum_col_names
    elif type == 'Gaussian Peaks': 
        return gaussian_col_names
    elif type == 'Peak Intensity': 
        return peak_col_names

In [20]:
select_type_of_intensity(type = 'Adjusted Voxel Sum')

['c3_voxel_sum_adjusted', 'c2_voxel_sum_adjusted', 'c1_voxel_sum_adjusted']

In [13]:
relevant_tracks = select_type_of_tracks(dataframe = filtered_tracks, only_dynamin_tracks = True, only_actin_tracks = False, 
                                        all_positive_tracks = False, both_tracks_positive = False)

In [14]:
updated_tracks = select_tracks_region_wise(dataframe = filtered_tracks, 
                                           tracks = relevant_tracks, 
                                           only_basal = True, only_apical = False, only_lateral = False, all = False)

In [15]:
updated_tracks

array([  536,   684,   838,  1001,  1428,  1586,  1589,  1742,  2623,
        2886,  2900,  3030,  3749,  3904,  3924,  4475,  4476,  4614,
        4774,  5292,  5336,  5465,  6661,  7790,  7957,  8203,  8744,
        8898,  8910,  9532, 10109, 10536, 10925, 10949, 12142, 12897,
       13644, 13885, 14760, 15095, 15109, 15607, 15847, 16094])

In [ ]:
ans = max_z_track_visualisation(track_of_interest = 22,zarr_array = zarr_arr ,main_tracking_df = track_df, 
                          channel = 2)

In [ ]:
plot_raw_movie(plot_type = 'max_z_slice', track_number = 22, raw_image = zarr_arr, main_tracking_df = track_df, channel = 2)

In [ ]:
plot_raw_movie(plot_type = 'total_z_sum', track_number = 22, raw_image = zarr_arr, main_tracking_df = track_df, channel = 2)

In [ ]:
plot_raw_movie(plot_type = 'max_intensity_projection', track_number = 22, raw_image = zarr_arr, main_tracking_df = track_df, channel = 2)